# Simple Backprop Example
### The Problem
Given a set of student GRE, GPA, uni rank, predict if a student will get admitted.

### The Theory
Build a simple neural network that updates the weights using gradient descent and backpropagation.

### How it works
1. Randomly initialise the weights using the random function
2. Run a forward pass through the neural network to generate a prediction
3. When a prediction is incorrect, we will update the weights
4. We run a backward pass to propagate the error back according to each node's contribution, and calculate delta w
5. Edit the weights using the calculated delta w value
6. Repeat steps 2-5 to train the network

In [19]:
import numpy as np
from data_prep import features, targets, features_test, targets_test

In [20]:
# seed the random number generator
np.random.seed(21)

In [21]:
# define the sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [22]:
# set hyperparameters

# number of hidden units
n_hidden = 2
epochs = 10000
learnrate = 0.005

In [23]:
# initialise values

# get number of records and deatures from the data
n_records, n_features = features.shape
# initialise last loss as None
last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

In [24]:
# run the network

# for each iteration
for e in range(epochs):
    # initialise Δw for both layers as zeros
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    # for each set of data
    for x, y in zip(features.values, targets):
        ## run a forward pass
        # input to the hidden layer
        hidden_input = np.dot(x, weights_input_hidden)
        # output from the hidden layer
        hidden_output = sigmoid(hidden_input)
        # input to output layer
        output_layer_in = np.dot(hidden_output, weights_hidden_output)
        # output layer out
        output = sigmoid(output_layer_in)

        ## run a backward pass to improve the neural net
        # calculate the prediction error
        error = y - output

        # calculate error term for the output unit
        output_error_term = error * output * (1 - output)

        ## propagate errors to hidden layer
        # calculate the hidden layer's contribution to the error
        hidden_error = np.dot(weights_hidden_output, output_error_term)
        
        # calculate the error term for the hidden layer
        hidden_error_term = hidden_error * (hidden_output * (1 - hidden_output))
        
        # update the Δw values
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:,None]

    # update weights using the Δw values
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # printing out the mean square error on the training set to see progress
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.27630002065852294
Train loss:  0.26234383325962973
Train loss:  0.25198020538763455
Train loss:  0.24434338417929377
Train loss:  0.23871681885367121
Train loss:  0.23455229005454342
Train loss:  0.23144711247759817
Train loss:  0.229111266798629
Train loss:  0.22733741993913803
Train loss:  0.22597737454942038


In [25]:
# run our test features through the neural network

# manually run the test features through the hidden layer
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
# manually run the output layer
out = sigmoid(np.dot(hidden, weights_hidden_output))

In [26]:
# calculate accuracy on test data

# if the prediction is > 50%, set as True
predictions = out > 0.5
# get mean of all the predictions
accuracy = np.mean(predictions == targets_test)
# print the prediction accuracy
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.750
